In [16]:
import dlib
import cv2
import imutils
from imutils.face_utils import FaceAligner
from imutils.face_utils import rect_to_bb
import numpy as np

In [15]:
def convert_to_numpy_array(shape, dtype='int'):
    cordinates = np.zeros((shape.num_parts,2),dtype=dtype)
    for i in range(shape.num_parts):
        cordinates[i] = (shape.part(i).x,shape.part(i).y)
    return cordinates

In [14]:
def align_face(face_img_orig,frontal_detector,shape_predict,gray):
    # Detect Face
    detected_face = frontal_detector(face_img_orig,1)

    # Initialize Face Alligner Object
    fa = FaceAligner(shape_predict,desiredFaceWidth=256)
    
    faceAligned = face_img_orig.copy()

    # Loop through detected images
    for i,d in enumerate(detected_face):
        (x, y, w, h) = rect_to_bb(d)
        faceOrig = imutils.resize(face_img_orig[y:y + h, x:x + w], width=256)
        faceAligned = fa.align(face_img_orig, gray, d)
    return faceAligned

In [50]:
def get_facial_landmark_points():
    # Read Image
    img = cv2.imread("8.jpg")

    #Resize Image
    img = cv2.resize(img,(500,500))
    
    # Image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)

    # Initialize Frontal Face Detector
    frontal_detector = dlib.get_frontal_face_detector()

    # 68 Landmark Points Marker
    shape_predict = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

    # Get aligned face
    img_aligned = align_face(img,frontal_detector,shape_predict,gray)
    
    # Copy aligned face
    img_aligned_cpy = img_aligned.copy()
    
    # Copy number 2
    img_aligned_cpy2 = img_aligned.copy()
    
    cv2.imwrite("dakotajohnson.jpg",img_aligned)
    
    # Points to list
    points = []
    
    #Detect from aligned face
    detected_face = frontal_detector(img_aligned,1)

    for i,d in enumerate(detected_face):
        x,y,w,h = rect_to_bb(d)
        # Shape predictor 
        shape = shape_predict(img_aligned,d)
        facial_cordinates = convert_to_numpy_array(shape)
#       print(facial_cordinates)
        img_croped = img_aligned[y:y + h, x:x + w]
        cv2.rectangle(img_aligned,(d.left(),d.top()),(d.right(),d.bottom()),(0,0,255),2)
        count = 1
        for (x,y) in facial_cordinates:
            cv2.circle(img_aligned,(x,y),2,(0,0,255),-1)
            points.append((x,y))
        
        # Save the cordinates of the points into a text file
        points_to_text(points)
        
        cv2.imshow("Original", img)
        cv2.imshow("Aligned", img_aligned_cpy)
        cv2.imshow("Facial Points Aligned",img_aligned)
        cv2.imshow("Cropped",img_croped)
        cv2.waitKey()
        cv2.destroyAllWindows()


In [51]:
def points_to_text(points):
    f = open("dakotajohnsonpoints.txt","w+")
    for p in points:
        f.write("%d %d\n"%(p[0],p[1]))

In [52]:
get_facial_landmark_points() 